# Amazon SageMaker Model Deployment using persistent endpoint for Real-Time inference

In this notebook we will depoy the Model that was trained using the preprocessed training_input from Data Wranger preprocessing Job.


### Perform model deployment using Amazon SageMaker

In [ ]:
import boto3
import sagemaker
import time
from sagemaker import get_execution_role, session
from time import gmtime, strftime
from sagemaker.model import Model
from sagemaker.image_uris import retrieve
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

# Setup defaults
region= boto3.Session().region_name
role = get_execution_role()

# Update this model_url with the trained model generated from training job that we ran as part of Data Wrangler
model_url = '<<Your S3 location of pre-trained model artifact>>'

# Pull the Amazon SageMaker built-in algorithm for xgboost
image_uri = retrieve('xgboost', region, '1.2-1')
# Create a model object using Model class
model = Model(image_uri=image_uri, model_data=model_url, role=role)

endpoint_name = 'DEMO-xgb-readmission-prediction-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

# Deploy the model as an endpoint hosted by Amazon SageMaker
hosted_endpoint = model.deploy(initial_instance_count=1,
                instance_type='ml.m4.xlarge',
                endpoint_name=endpoint_name)

### Perform inference against the deployed model


In [ ]:
# Create SageMaker Predictor object to pass in the test data for inference
predictor = Predictor(endpoint_name=endpoint_name, serializer=CSVSerializer())

print("Sending test traffic to the endpoint {}. \nPlease wait...".format(endpoint_name))

with open("test_data/test_data_UCI_sample.csv", 'r') as f:
    for row in f:
        payload = row.rstrip('\n')
        print('Incoming Payload => ', payload)
        response = predictor.predict(data=payload).decode('utf-8')
        print("Inference Output => ", response)
        time.sleep(1)
        
print("Done!")